# Artificial Intelligence
## L.EIC – 3rd Year/2nd Semester
### Exercise Sheet 1
# Solving Problems by Searching

## The Two Buckets Problem

<img src="https://qph.cf2.quoracdn.net/main-qimg-45726b16b460cae0147ae8ca245a8fb0-pjlq" width="250px" height="250px" align="right">

Two buckets of capacities **c1** (e.g. 4 liters) and **c2** (e.g. 3 liters), respectively, are initially empty. The buckets do not have any intermediate markings. The only operations you can perform are:

- Fill (completely) a bucket
- Empty a bucket.
- Pour one bucket into the other (until the target one is full or the source one is empty).

The aim is to determine which operations to carry out so that the first bucket contains exactly **n** liters (e.g. 2 litres).

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.


### State Representation

> Indented block


B1, B2

### Initial State
B1 = 0
B2 = 0

### Operators
Name | Pre Conditions | Effects | Cost
--- | --- | --- | ---
Empty1 | W1 > 0 | W1 = 0 | 1
Empty2 | W2 > 0 | W2 = 0 | 1
fill1 | W1 < C1 | W1 = C1 | 1
fill2 | W2 < C2 | W2 = C2 | 1
pour12_fill2 |
pour12_empty1 |
pour21_fill1 |
pour21_empty2 |

### Objective Test

W1 = 2


What is the size of the state space for this problem? Represent the state space by drawing the possible objective states and displaying some of the possible transitions from the initial state.

Solve the problem by hand, using tree search. What solutions have you found?

### Building a computational approach to handle the problem

To build a program to solve the buckets problem, we will implement a solution that separates the problem definition from the algorithms used to traverse the state space. This way, we can reuse our implementations of the search strategies in other problems.

#### Representing the two buckets problem as a search problem

Let's start by defining a state for the buckets problem. For that, it'll suffice to aggregate two quantities, each representing the amount of water in one of the buckets. We also define a way of printing the state.

In [ ]:
class BucketState:
    c1 = 4   # capacity for bucket 1
    c2 = 3   # capacity for bucket 2

    def __init__(self, b1, b2):
        self.b1 = b1
        self.b2 = b2

    '''needed for the visited list'''
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)

    def __hash__(self):
        return hash((self.b1, self.b2))
    ''' - '''

    def __str__(self):
        return "(" + str(self.b1) + ", " + str(self.b2) + ")"

: 

Now we define each of the operators on states:

In [ ]:
# emptying the first bucket
def empty1(state):
    if state.b1 > 0:
        return BucketState(0, state.b2)
    return None

# emptying the second bucket
def empty2(state):
    # your code here
    if state.b2 > 0:
        return BucketState(state.b1, 0)
    return None


# your code here
def fill1(state):
    if state.b1 < state.c1:
        return BucketState(state.c1, state.b2)
    return None

def fill2(state):
    if state.b2 < state.c2:
        return BucketState(state.b1, state.c2)
    return None

def pour12_fill2(state):
    if state.b2 + state.b1 >= state.c2 and state.b1 > 0 and state.b2 < state.c2:
        return BucketState(state.b1 - (state.c2 - state.b2), state.c2)
    return None

def pour12_empty1(state):
    if state.b2 + state.b1 < state.c2 and state.b1 > 0:
        return BucketState(0, state.b2 + state.b1)
    return None

def pour21_fill1(state):
    if state.b1 + state.b2 >= state.c1 and state.b2 > 0 and state.b1 < state.c1:
        return BucketState(state.c1, state.b2 - (state.c1 - state.b1))
    return None

def pour21_empty2(state):
    if state.b1 + state.b2 < state.c1 and state.b2 > 0:
        return BucketState(state.b1 + state.b2, 0)
    return None

: 

The following function will aggregate all states that can be generated from a given one:

In [ ]:
def child_bucket_states(state):
    new_states = []
    if(empty1(state)):
        new_states.append(empty1(state))
    if(empty2(state)):
        new_states.append(empty2(state))
    if(fill1(state)):
        new_states.append(fill1(state))
    if(fill2(state)):
        new_states.append(fill2(state))
    if(pour12_fill2(state)):
        new_states.append(pour12_fill2(state))
    if(pour12_empty1(state)):
        new_states.append(pour12_empty1(state))
    if(pour21_fill1(state)):
        new_states.append(pour21_fill1(state))
    if(pour21_empty2(state)):
        new_states.append(pour21_empty2(state))
    return new_states

: 

Play around with the state transition operators and check if they are working properly:

In [ ]:
s = BucketState(0, 0)
s = fill1(s)
print(s)

child_bucket_states(BucketState(0, 0))

# your code here


: 

Finally, we need to define the goal condition:

In [ ]:
def goal_bucket_state(state):
    # your code here
    if state.b1 == 2:
        return True
    return False


: 

Test your goal condition:

In [ ]:
# your code here
a = BucketState(0, 0)
b = BucketState(2, 0)
print(goal_bucket_state(a))
print(goal_bucket_state(b))

: 

#### Implementing search algorithms

Let us start by defining an appropriate structure to represent a node in a search tree. Each tree node will include:
- a state of the problem
- a link to its parent (to allow traveling from a leaf node towards the root of the tree)
- a list of child nodes

In [ ]:
# A generic definition of a tree node holding a state of the problem
class TreeNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self

: 

##### Breadth-first search

Based on this structure, we can now implement breadth-first search. Note that we want the implementation to be independent of the problem at hand (in this case, the two buckets problem).

In [ ]:
from collections import deque

def breadth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state)   # create the root node in the search tree
    queue = deque([root])   # initialize the queue to store the nodes

    while queue:
        node = queue.popleft()   # get first element in the queue
        if goal_state_func(node.state):   # check goal state
            return node

        for state in operators_func(node.state):   # go through next states
            # create tree node with the new state
            # your code here
            newNode = TreeNode(state)

            # link child node to its parent in the tree
            # your code here
            node.add_child(newNode)

            # enqueue the child node
            # your code here
            queue.append(newNode)


    return None

: 

We can now use this function to actually perform a breadth-first search on the buckets problem: we pass it the initial state, our goal condition function, and the function for obtaining child states.

In [ ]:
goal = breadth_first_search(BucketState(0,0),
                            goal_bucket_state,
                            child_bucket_states)
print(goal.state)

: 

In order to print the actual steps from the initial state to the last, we can take advantage of each node's link to its parent.

In [ ]:
from collections import deque

def print_solution(node):
    # your code here
    newNode = node
    count = 0
    nodeList = [newNode]
    while newNode.parent:
      count += 1
      nodeList.append(newNode.parent)
      newNode = newNode.parent

    print(f"Found goal state in {count} steps:")
    for node in nodeList[::-1]:
      print(node.state)

    return

: 

Now we can print the solution:

In [ ]:
print_solution(goal)

: 

If we need a description for each of the employed operators, we could have each operation function return also such a description, and modify the TreeNode class so that each node also includes a description of the edge to get there. We leave that as an exercise after class.

##### Depth-first search

Implement depth-first search (again, in a manner that is independent of the problem at hand). You can start from your breadth-first search implementation and with minor changes get an implementation for depth-first search.

In [ ]:
def depth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state)
    stack = [root]
    visited = set()

    while stack:
        node = stack.pop()
        if goal_state_func(node.state):
            return node

        visited.add(node.state)

        for state in operators_func(node.state):
            if state not in visited:

                newNode = TreeNode(state)

                node.add_child(newNode)

                stack.append(newNode)

                visited.add(state)
    return None

: 

Test it on the two buckets problem.

In [ ]:
# your code here
goal = depth_first_search(BucketState(0,0),
                            goal_bucket_state,
                            child_bucket_states)
print_solution(goal)

: 

If you are unable to get a solution, think about it: depth-first search is not a complete search method, and one of the reasons for that is if the state space contains cycles. As such, you need to make sure you avoid entering into a cycle by keeping a visited nodes list or set and checking that list whenever you generate a new state.

##### Depth-limited search

Another way to make it work is to impose a depth limit to the problem. Implement depth-limited search.

In [ ]:
def depth_limited_search(initial_state, goal_state_func, operators_func, depth_limit):
    root = TreeNode(initial_state)
    stack = [(root, 0)]

    while stack:
        node, depth = stack.pop()
        if depth > depth_limit:
            continue

        if goal_state_func(node.state):
            return node

        if depth < depth_limit:
            for state in operators_func(node.state):

                newNode = TreeNode(state)

                node.add_child(newNode)

                stack.append((newNode, depth + 1))

    return None

: 

Test it on the two buckets problem.

In [ ]:
# your code here
goal = depth_limited_search(BucketState(0,0),
                            goal_bucket_state,
                            child_bucket_states,
                            10)
print_solution(goal)

: 

##### Iterative deepening search

Based on depth-limited, you can easily implement iterative-deepening search.

In [ ]:
def iterative_deepening_search(initial_state, goal_state_func, operators_func, depth_limit):
    for depth_limit in range(depth_limit + 1):
        result = depth_limited_search(initial_state, goal_state_func, operators_func, depth_limit)
        if result is not None:
            return result
    return None


: 

Again, test it on the two buckets problem.

In [ ]:
# your code here
goal = iterative_deepening_search(BucketState(0,0),
                            goal_bucket_state,
                            child_bucket_states,
                            10)
print_solution(goal)

: 

## The Missionaries and Cannibals Problem

<img src="https://allfish24.files.wordpress.com/2016/09/gambar-1.jpg">

Three missionaries and three cannibals are on one of the banks of the river with a boat that only takes one or two people. The boat cannot travel the river alone.

The goal is to find a way to get the six to the other bank of the river without ever leaving more cannibals than missionaries on one of the banks (even at the instant they leave/join the boat) during the process.

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

Solve the problem by hand, using tree search. What solutions have you found?

Represent the problem as a search problem and take advantage of the implemented search algorithms to find solutions!

In [ ]:
# your code here


: 